In [1]:
import pandas as pd

In [2]:
# reading the data
file_name = 'NSE:RELIANCE-EQ 2020-08-01 to 2021-08-03.csv'
resample_interval = '5T'
# NOTE: inorder to group the data by date we need to remove our date column as index
# if we set it as index, for some reason we can't access it like data['Date']
data = pd.read_csv(file_name)

data.tail()

,Count,Date,Open,High,Low,Close,Volume
93657,93657,2021-08-03 15:25:00,2089.70,2090.00,2088.90,2089.50,99576
93658,93658,2021-08-03 15:26:00,2089.25,2089.95,2088.75,2089.20,41469
93659,93659,2021-08-03 15:27:00,2089.50,2090.80,2089.10,2090.20,40300
93660,93660,2021-08-03 15:28:00,2090.95,2091.00,2089.15,2090.00,27096
93661,93661,2021-08-03 15:29:00,2090.30,2090.85,2089.50,2090.45,11527


In [3]:
print("Length of original data: ", len(data))

Length of original data:  93662


In [4]:
# NOTE: There is a problem while resampling the data
# Pandas treats the Date column as a continuous time value
# where as our data is discrete, 9:15 to 3:30 and back to 9:15

# So, in order to solve this, we group the data by date and create n new dataframes
# where n is the number of days in the data
# and then we resample each of these dataframes

In [5]:
# Creating a new column with the date only, this is used for grouping
# And we apply resampling for each group seperated by day

# converting the date column from string to datetime type for creating a date column
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d %H:%M:%S")
data['day'] = data.apply(lambda x: x.Date.date(), axis=1)
data

,Count,Date,Open,High,Low,Close,Volume,day
0,0,2020-08-03 09:15:00,2051.30,2054.00,2035.20,2038.50,422003,2020-08-03
1,1,2020-08-03 09:16:00,2038.15,2044.40,2038.15,2041.85,192419,2020-08-03
2,2,2020-08-03 09:17:00,2041.00,2042.40,2038.95,2040.50,149905,2020-08-03
3,3,2020-08-03 09:18:00,2041.00,2041.50,2035.00,2040.75,183827,2020-08-03
4,4,2020-08-03 09:19:00,2040.75,2040.75,2037.70,2039.50,93165,2020-08-03
...,...,...,...,...,...,...,...,...
93657,93657,2021-08-03 15:25:00,2089.70,2090.00,2088.90,2089.50,99576,2021-08-03
93658,93658,2021-08-03 15:26:00,2089.25,2089.95,2088.75,2089.20,41469,2021-08-03
93659,93659,2021-08-03 15:27:00,2089.50,2090.80,2089.10,2090.20,40300,2021-08-03
93660,93660,2021-08-03 15:28:00,2090.95,2091.00,2089.15,2090.00,27096,2021-08-03


In [6]:
# Setting the date column as index, because for resampling we need to set the index as date
data.set_index("Date", inplace=True)
data

,Count,Open,High,Low,Close,Volume,day
Date,,,,,,,
2020-08-03 09:15:00,0,2051.30,2054.00,2035.20,2038.50,422003,2020-08-03
2020-08-03 09:16:00,1,2038.15,2044.40,2038.15,2041.85,192419,2020-08-03
2020-08-03 09:17:00,2,2041.00,2042.40,2038.95,2040.50,149905,2020-08-03
2020-08-03 09:18:00,3,2041.00,2041.50,2035.00,2040.75,183827,2020-08-03
2020-08-03 09:19:00,4,2040.75,2040.75,2037.70,2039.50,93165,2020-08-03
...,...,...,...,...,...,...,...
2021-08-03 15:25:00,93657,2089.70,2090.00,2088.90,2089.50,99576,2021-08-03
2021-08-03 15:26:00,93658,2089.25,2089.95,2088.75,2089.20,41469,2021-08-03
2021-08-03 15:27:00,93659,2089.50,2090.80,2089.10,2090.20,40300,2021-08-03


In [7]:
d = {'Open':'first', 'High':'max','Low':'min','Close':'last','Volume':'sum'}

day_based_groups = data.groupby('day')
day_based_groups_resampled = []

for date, group in day_based_groups:
    # the date based group has already index as date
    group_resampled = group.resample(resample_interval, origin='start').agg(d)
    day_based_groups_resampled.append(group_resampled)

In [8]:
# concatenating all the dataframe groups (day based) into one dataframe
resampled_final_data = pd.concat(day_based_groups_resampled)

# adding Count column back to the dataframe, because it got removed while resampling
index = pd.Index(range(len(resampled_final_data)))
resampled_final_data.reset_index(inplace=True)
resampled_final_data = resampled_final_data.set_index(index, )
resampled_final_data.index.name = 'Count'

resampled_final_data.head()

,Date,Open,High,Low,Close,Volume
Count,,,,,,
0,2020-08-03 09:15:00,2051.3,2054.00,2035.00,2039.50,1041319
1,2020-08-03 09:20:00,2039.6,2046.90,2036.00,2043.40,634220
2,2020-08-03 09:25:00,2043.4,2043.40,2037.00,2041.75,310635
3,2020-08-03 09:30:00,2041.5,2041.75,2032.30,2040.45,493650
4,2020-08-03 09:35:00,2040.2,2044.85,2039.85,2043.00,272013


In [9]:
resampled_final_data.to_csv(f"{file_name[:-4]}_{resample_interval}.csv")